# Animation

Animations can be an effective way to show an extra dimension of your data, especially changes over time. Clearly, they're not appropriate everywhere, eg in static media! For various reasons, they should be used sparingly, but-used rarely and with care-they can convey a message about data more effectively than a static chart ever would. Perhpas the most famous example of an animated chart is the legendary Hans Roslings' [gapminder animation](https://youtu.be/jbkSRLYSojo?t=29) covering 200 countries and 200 years of progress in income and health.

The most common scenario for wanting to create an animation is sharing on social media (gifs are ubiquitous on the internet) or displaying an animation of some of your results during a talk. In both these cases, creating a gif is probably the best and safest option. 'The best' because although video files tend to be very large (and thus difficult to share), gifs can be turned into small files, with a bit of tweaking. 'The safest' because even though presentation software like Microsoft's Powerpoint can crumble when trying to play an embedded video file, gifs usually work just fine. And while video formats differ across operating systems, all of them will play gifs (all you need to play a gif is a browser such as Safari or Chrome).  However, you can also create animations in other video formats, such as .mp4.

There are several ways to produce animations in Python. 

## Plot To Animate 

First, we need a plot to animate.


## Creating Animations Using the Gif Package

